In [1]:
import pandas as pd
import os
from modules.Date2 import find_starting_point, find_specific_date, convert_date
from modules.variables2 import *
from modules.Data_Quality import *
from modules.Cost_centre_structure import *

In [2]:
end_month = 'Apr'
end_year = '2019/20'

files_info =[]
dataframes = []
root = './'
input_folder = root+'Input files/' +'Infrastructure/' + '2019'
output_folder = root+'Prepared Files/' +'Infrastructure/' + '2019'
combined_folder = root+'Combined files/' +'Infrastructure/' + '2019/'
directory = os.fsencode(input_folder)
num = 0

In [3]:
def process_file(file_path, division):
    global num
    xls = pd.ExcelFile(file_path)
    subregion_dict = division_to_subregion.get(division)

    if not subregion_dict:
        print(f"No subregion dictionary found for division {division}")
        return

    for sheet_name in xls.sheet_names:
        if any(sheet_name in cost_centres for cost_centres in subregion_dict.values()):
            num += 1
            # Load the data from the sheet, specifying no header to properly identify the row
            data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
            start_month, start_year, row, start_col = find_starting_point(data)
            end_row, end_col = find_specific_date(data, end_month, end_year)
            dates = convert_date(data, row, start_col, end_col)
            print(division, sheet_name)
            extracted_data = extract_var_columns(data, start_col, end_col)
            extracted_data, region, division, cost_centre = get_metadata(extracted_data, division, sheet_name)
            final_df = pd.DataFrame({'Date': dates})
            final_df = pd.concat([final_df, extracted_data.reset_index(drop=True)], axis=1)
            num_col = len(final_df.columns)
            has_missing, rows_with_missing = check_missing_values(final_df, exclude_column)
            has_secured_missing = check_missing_secured(final_df)
            duplicated = check_duplicates(final_df)
            #incorrect_relationship = check_relationships(final_df)

            var_name = f"{os.path.basename(file_path)}_{sheet_name}_missing"
            globals()[var_name] = rows_with_missing

            file_info = {'file_name': os.path.basename(file_path).split('.')[0],
                         'num': num,
                         'region': region,
                         'division': division,
                         'cost_centre': cost_centre,
                         'num_col': num_col,
                         'Start month': start_month,
                         'Start year': start_year,
                         'row': row,
                         'End column': end_col,
                         'missing': has_missing,
                         'secured missing': has_secured_missing,
                         'duplicated': duplicated,
                         #'incorrect_relationship': incorrect_relationship
                         }

            files_info.append(file_info)
            dataframes.append(final_df)
            output_file = f"{division}_{sheet_name.replace(' ', '_')}"
            prepared_file_path = os.path.join(output_folder, f'{output_file}_ok.csv')
            final_df.to_csv(prepared_file_path, index=False)

In [5]:
# Main loop to process all files
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") or filename.endswith(".xls") or filename.endswith(".xlsm"):
        file_path = os.path.join(input_folder, filename)

        # Extract division from filename
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        name_parts = base_name.split('-')
        if len(name_parts) > 1:
            division = name_parts[1].strip().lower()
            process_file(file_path, division)

files_info_df = pd.DataFrame(files_info)
files_info_df.to_csv('files_info_INF_2019_1.csv', index=False)

infrastructure CM Scotland
infrastructure PM Scotland
infrastructure PMO Scotland
infrastructure CM NW
infrastructure PM NW
infrastructure PMO NW
infrastructure CM YNE
infrastructure PM YNE
infrastructure PMO YNE
infrastructure CM SE
infrastructure PM SE
infrastructure PMO SE
infrastructure MOD
infrastructure PA
infrastructure Co


In [67]:
len(dataframes)

16

In [57]:
# Filter and combine DataFrames to include only 'Advisory' and 'AMCL' divisions
included_divisions = ['advisory', 'amcl']

filtered_dataframes1 = [
    df for df in dataframes
    if df['Division'].iloc[0].lower() in included_divisions
]

In [11]:
excluded_divisions = []  # Add the names of divisions to exclude
excluded_cost_centres = ['PM_RE_Digital', 'CM_alinea', 'INF_Def_Nth', 'INF_Def_SE','INF_T&U_SW',
                         'INF_Def_CM_SW','INF_Def_P3M_SW', 'INF_Digital', 'PM_Pcon_Sth', 'PM_Co', 'AMCL_IES', 'ADV_NZ','ADV_SHQTel', 'PM_Project_Controls', 'CM_Co', 'CM_Cambridge', 'INF_Co', 'INF_PMO_Scotland', 'ADV_Disp', 'ADV_Proc']  # Add the names of cost centres to exclude

# Filter dataframes to exclude problematic divisions and cost centres
filtered_dataframes = []
for df in dataframes:
    if df['Division'].iloc[0].lower() not in excluded_divisions and df['Cost_Centre'].iloc[0] not in excluded_cost_centres:
        filtered_dataframes.append(df)

In [12]:
len(filtered_dataframes)

13

In [13]:
combined_df = pd.concat(filtered_dataframes, ignore_index=True)

In [14]:
combined_df.rename(columns={'secured':'income_ex'}, inplace=True)
columns_to_keep = [
    'Date','income_ex', 'direct_costs', 'overheads',
    'total_headcount', 'Region', 'Cost_Centre', 'Division'
]
combined_df = combined_df[columns_to_keep]

In [22]:
# Save the combined dataframe to a CSV file
combined_csv_path = combined_folder + 'combined_file_INF_2019.csv'
combined_df.to_csv(combined_csv_path, index=True)

excluded_dataframes = [df for df in dataframes if df['Division'].iloc[0] in excluded_divisions or df['Cost_Centre'].iloc[0] in excluded_cost_centres]
for df in excluded_dataframes:
    print(f"Excluded Division: {df['Division'].iloc[0]}, Cost Centre: {df['Cost_Centre'].iloc[0]}")

Excluded Division: Infrastructure, Cost Centre: INF_PMO_Scotland
Excluded Division: Infrastructure, Cost Centre: INF_Co


In [16]:
for index, row in combined_df.iterrows():
    if row.isna().any(): 
        print(f"Cost_Centre value for this row: {row['Cost_Centre']}")

In [17]:
duplicated_rows = combined_df[combined_df.duplicated(keep=False)]

if not duplicated_rows.empty:
    print("Rows with duplicated values:")
    for index, row in duplicated_rows.iterrows():
        print(f"Row index {index} has duplicated values.")
        print(f"Cost_Centre value for this row: {row['Cost_Centre']}")
    else:
        print("No duplicated values found in the DataFrame.")

In [10]:
# Example usage
sheet_name = 'PM Scotland'
file = input_folder + '/P1 -  Infrastructure.xlsx'
df = pd.read_excel(file, sheet_name=sheet_name)
start_month, start_year, row, start_col = find_starting_point(df)
end_row, end_col = find_specific_date(df, end_month, end_year)
dates = convert_date(df, row, start_col, end_col)
print(dates)

DatetimeIndex(['2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01'],
              dtype='datetime64[ns]', freq=None)


In [21]:
combined_df[combined_df['Cost_Centre']== 'INF_PM_Scotland'].Date.info()

<class 'pandas.core.series.Series'>
Index: 12 entries, 12 to 23
Series name: Date
Non-Null Count  Dtype         
--------------  -----         
12 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 192.0 bytes
